In [1]:
## Bring in needed mods
import glob

## Set file to drop
## We do not have the matching file for this samples
drop_file = '../FASTQ/JW-S1_S1_L001_R2_001.fastq.gz'

## Load in fastq file names
file_paths_r1 = sorted(glob.glob('../FASTQ/*R1*.fastq.gz'))
file_paths_r2 = sorted([a for a in glob.glob('../FASTQ/*R2*.fastq.gz') if a != drop_file])

## Set referance path
refpath = '../REFERENCE/XL280_NP_NUCLEAR_FINAL_July2020.fasta'

In [2]:
## Initate line
lines = []

## Append mkdir commands
lines.append('mkdir -p ../BAMS\nmkdir -p ../DATA\n\n')

## index the reference file
lines.append('bwa index ../REFERENCE/XL280_NP_NUCLEAR_FINAL_July2020.fasta\n\n')

## Iterate thru R1 path
for f1 in file_paths_r1:

    ## Set path to R2
    f2 = 'R2'.join(f1.split('R1'))
 
    ## Check work
    assert f2 in file_paths_r2, 'Missing fastq file: %s'%f2

    ## Set the out bam name and path
    outbam =  '../BAMS/'+f1.split('/')[-1].split('R1')[0]+'.bam'

    ## Write a the bwa command
    bwaline = 'bwa mem -a -M %s %s %s | samtools view -b | samtools sort -o %s\n\nsamtools index %s\n\n\n'%(refpath,f1,f2,outbam,outbam)
 
    ## Append the bwa line
    lines.append(bwaline)
    
## Append bam listing to file
lines.append('ls -d "$PWD"/../BAMS/*.bam > ../DATA/bamlist.txt\n\n')

## Append sam tools command
lines.append('samtools depth -f ../DATA/bamlist.txt > ../DATA/SampleDepths.tsv\ngzip ../DATA/SampleDepths.tsv\n\n')

## Open file for writting bwa file too
fout = open('./bwa_XL280np.sh','w')

## Write to file, and close the file.
fout.writelines(lines);fout.close()